In [1]:
import pandas as pd
import numpy as np

In [2]:
inputdbf = "BGDAdmin3_ProxTable.csv"
inputpop = "Upazila populations.xlsx"
operpop_sheet = 'Output'

lambder_list = [1000,100,50,25,10,5,2,1,0.5,0.25]

In [3]:
distmatrix = pd.read_csv(inputdbf)
distmatrix.sample(5)

,OBJECTID,IN_FID,NEAR_FID,NEAR_DIST,NEAR_RANK
191011,0,363,471,0.826694,68
214395,0,407,268,1.366840,308
206082,0,391,265,1.834582,412
116296,0,221,24,0.461348,47
128170,0,243,228,2.153272,350


In [4]:
pop_wb_ws = pd.read_excel(inputpop,operpop_sheet)
pop_wb_ws.sample(5)

,IN_FID,NAME,POPULATION
41,46,Sarail,315208
235,258,Companiganj,250579
232,255,Balaganj,320227
399,439,Dhanbari,176068
241,265,Adamdighi,195186


In [5]:
distmatrix["pop_in"] = distmatrix.IN_FID.replace(pop_wb_ws.set_index("IN_FID").POPULATION)
distmatrix["pop_near"] = distmatrix.NEAR_FID.replace(pop_wb_ws.set_index("IN_FID").POPULATION)

In [6]:
distmatrix.sample(5)

,OBJECTID,IN_FID,NEAR_FID,NEAR_DIST,NEAR_RANK,pop_in,pop_near
58710,0,111,359,1.643031,321,263608,133650
128303,0,243,476,2.815108,483,197997,221163
241886,0,459,311,2.560212,446,345179,206788
98297,0,186,492,2.661615,459,259648,119084
37314,0,70,481,5.327614,495,264389,365103


In [7]:
def market_access(x,lambdar):
    return sum(x.pop_near*np.exp(-lambdar*x.NEAR_DIST))+x.pop_in.iloc[0]

In [8]:
output = pd.DataFrame()
for lamdar in lambder_list:
    output[lamdar] = distmatrix.groupby("IN_FID").apply(lambda x:market_access(x,lamdar))

In [9]:
output.sample(10)

,1000.0,100.0,50.0,25.0,10.0,5.0,2.0,1.0,0.5,0.25
IN_FID,,,,,,,,,,
47,2068129.741044,2181559.850815,2271276.251551,2666475.275680,4852201.477853,10442082.892447,28948118.813141,53611930.875662,82053741.289779,1.059480e+08
136,1696029.404073,1809155.207641,2113665.646449,2934099.611543,6392120.317569,12869264.251553,31378198.170702,55930274.729375,84072782.835884,1.073934e+08
274,1938300.000000,1944446.440683,2024306.153673,2346123.528451,3763729.272115,7038766.208078,20359854.511285,42635744.188422,72010130.307050,9.880050e+07
145,3484560.000000,3487875.508135,3534552.156492,3798988.874923,5179970.071385,8496362.852824,23405552.557385,47877151.939857,77924789.874266,1.035071e+08
386,1822114.417415,2086321.649889,2286521.813502,2619998.538331,3940933.732788,7601372.894968,23419486.608701,47632261.887604,77131185.715591,1.026530e+08
85,465768.000000,467788.386201,475725.914721,504037.187383,799921.362063,1917402.006403,8080204.939171,23452171.148397,51295401.235634,8.252743e+07
249,1764909.000000,1765125.530152,1775974.423379,1878411.157043,2561505.248286,4426010.706127,14296484.719211,34863119.223700,65213076.905493,9.425048e+07
77,511902.003858,538825.585686,586797.424749,707040.853502,1261964.828449,2755129.131289,10474145.415319,27830885.226288,56494417.926957,8.682123e+07
246,1512484.000000,1512484.750411,1513414.443798,1550081.115587,2044981.191713,3589174.569708,12034955.138715,31285305.932136,61465837.198782,9.139810e+07


In [10]:
output.to_csv("output_with_pandas.csv")